# **Install and Import Libraries**

> ##### **create .streamlit/secrets.toml file inside notebooks folder with following lines:**

> ###### **OPENAI_API_KEY = "<api_key>"**


In [ ]:
%pip install -r ../requirements.txt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

import sys
sys.path.append('../src')
from data_pipeline import *

# **1. Scrape Website**

In [ ]:
df = scrape_website()

In [ ]:
df

# **2. Download all PDFs from links**

In [ ]:
df = download_documents()

In [ ]:
df

# **4. Extract HTML from PDFs**

In [ ]:
convert_pdf_to_html()

# **4. Extract Meeting Metadata from PDF with LLM**

In [ ]:
df = extract_meeting_metadata()

In [ ]:
df

# **5. Extract Proposals and Decisions**

### still experimenting

In [ ]:
non_agenda_titles = [
    'Sammanträdets laglighet och beslutsförhet',
    'Godkännande av föredragningslistan',
    'Val av protokolljusterare',
    'Sammanträdets laglighet och beslutförhet',
    'Sammanträdets konstituerande',
    'Kokouksen laillisuus ja päätösvaltaisuus',
    'Kahden pöytäkirjantarkastajan valinta',
    'Esityslistan hyväksyminen',
    'Val av protokolljusterare och protokollförare'
]

In [ ]:
# filter to only agenda items documents
filtered_df = df[~df['rubrik'].isin(['Beslutande', 'Sammanträdesuppgifter och deltagande', 'Kokoustiedot ja osallistujat', 'Vln:Beslutande', 'Päättäjät', *non_agenda_titles])]
filtered_df = filtered_df[(filtered_df['parent_link'] == "") & (~filtered_df['section'].isin(["", "§ 0"]))]

In [ ]:
filtered_df

In [ ]:
stadsfullmäktige_df = filtered_df[filtered_df['verksamhetsorgan'] == 'Stadsfullmäktige']
stadsfullmäktige_df

In [ ]:
df.verksamhetsorgan.unique()

In [ ]:
import fitz
import docx
from pprint import pprint
from tqdm import tqdm
import re

# get unique organs
organs = df.verksamhetsorgan.unique()

# Construct the alternation group for organ names
organ_alternation_group = '|'.join(re.escape(organ) for organ in organs)

# Assemble the full regex pattern
reference_regex = rf'({organ_alternation_group}) (\d{{1,2}}\.\d{{1,2}}\.\d{{4}}),? (§ \d+|\d+ §)'


# Compile the regex pattern
reference_compiled_regex = re.compile(reference_regex)

print(reference_compiled_regex)


for index, row in stadsfullmäktige_df.iterrows():
  date = row['meeting_date'].replace('.', r'\.')
  match_section = rf"(?:(?:{row['verksamhetsorgan']} {date},? (?:{row['section']}|{row['section'].replace('§','').strip()} §)))(?:(?!(?:{row['verksamhetsorgan']} {date},? (?:{row['section']}|{row['section'].replace('§','').strip()} §))).)*$"
  filepath = f'/content/drive/MyDrive/all_protocols/protocols/{row["doc_name"]}'
  print(match_section)
  print(row['doc_link'], row['verksamhetsorgan'], row['meeting_date'], row['section'])
  try:
    with fitz.open(filepath) as doc:
      text = "".join(page.get_text("text", flags=~fitz.TEXT_PRESERVE_IMAGES) for page in doc)
  except:
    doc = docx.Document(filepath)
    text = '\n'.join(para.text for para in doc.paragraphs)

  matches = reference_compiled_regex.findall(text)
  print("\n".join(set([match for match in [" ".join(match) for match in matches]])))
  print("-"*100)

  # Find the match
  match = re.search(match_section, text, re.DOTALL)

  if match:
      print("Captured Text:", match.group(0))
  # print(text)
  print("="*100)